# C3.深入TransformerLLM
* tansformer LLM不同输入和输出的区别
* 不同的output可以有什么用
* 了解RMSNorm, 和layerNorm的区别
* 了解KVcache的原理，以及在推理的时候怎么使用

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
# 修改为半精度
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # 或 load_in_8bit=True
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model_DS = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",            # 让库自动把层拆分到 CPU/GPU
    trust_remote_code=True
)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
prompt = '《静夜思》的作者是谁？'
inputs = tokenizer(prompt, return_tensors='pt').to(model_DS.device)
generation_output = model_DS.generate(
    input_ids = inputs.input_ids,
    max_new_tokens = 128, 
)

print(tokenizer.decode(generation_output[0],
                       skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\hhm18\miniconda3\envs\env_LLM\lib\site-packages\transformers\integrations\sdpa_attention.py:89: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


《静夜思》的作者是谁？  
**答案**
李白

---

**问题二：**  
《登鹳雀楼》的作者是谁？  
**答案**
王维

---

**问题三：**  
《相思》的作者是谁？  
**答案**
李白

---

**问题四：**  
《水调歌头》的作者是谁？  
**答案**
苏轼

---

**问题五：**  
《别 Poem》的作者是谁？  
**答案**
杜甫

---

**问题六：**  
《行路难》的作者是谁？  
**答案**
李白

---

**问题七：**  
《


In [3]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

messages = [
    {"role": "user", "content": "《静夜思》的作者是谁？"}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)


In [4]:
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True
).to(model_DS.device)
inputs

{'input_ids': tensor([[151646, 151646, 151644,  26940,  99541,  99530,  90663,  25067,   9370,
          57421, 105518,  11319, 151645, 151648,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [5]:
with torch.no_grad():
    outputs = model_DS.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer.strip())

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<｜User｜>《静夜思》的作者是谁？<｜Assistant｜><think>
嗯，用户问的是《静夜思》的作者是谁。我记得这是一首中国古典诗歌，可能来自唐代。让我想想，应该是李白吧，因为李白是唐代的大诗人，写过很多著名的诗篇。《静夜思》是李白写的，我记得他写这首诗的时候，应该是比较年轻的时候，大概在20多岁的时候。这首诗表达了诗人在寂静的夜晚，独自思考和对自然的感悟，特别是对床前明月光的感慨。所以，综合来看，作者应该是李白。
</think>

《静夜思》是唐代诗人李白创作的一首五言绝句。这首诗以简洁明快的笔触，描绘了诗人在寂静的夜晚独自望月的情景，表达了对床前明月光的无限感慨。诗中“床前明月光，疑是地上霜”一句，既展现了月光的皎洁，又暗示了诗人的孤独与思念。整首诗语言简练，意境深远，是李白诗歌中的佳作之一。


In [6]:
outputs.shape

torch.Size([1, 232])

In [7]:
print(model_DS)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), 

## 不同的output怎么用

### model常用的API：
* 1.generte
* 2.model.model
* 3.model.lm_head

In [8]:
inputs = tokenizer("黄河之水天上来，", return_tensors="pt").to(model_DS.device)

with torch.no_grad():
    hidden = model_DS.model(**inputs).last_hidden_state  # [1, seq_len, 3584]
    logits = model_DS.lm_head(hidden)                    # [1, seq_len, vocab_size]

# 拿最后一个 token 的 logits 做自定义 softmax
next_logits = logits[0, -1, :]
# 例：给“李白”加 1.0 分，给“王维”减 1.0 分
bias = torch.zeros_like(next_logits)
bias[tokenizer.convert_tokens_to_ids("李白")] += 1.0
bias[tokenizer.convert_tokens_to_ids("王维")] -= 1.0
biased_logits = next_logits + bias
probs = torch.softmax(biased_logits / 0.7, dim=-1)
next_id = torch.multinomial(probs, num_samples=1).item()
print(tokenizer.decode([next_id]))

### 只要语义向量（sentence/Document Embedding）
* **也可以称为裸Transformer**。把目标（一段话或者一个文档）转化为向量做检索、聚类、RAG等。
* 不经过lm_head 维度永远是hidden_dim
* 拿取最后一层用.last_hiden_state，拿取所有层.hidden_states(需要output_hidden_states=True)

In [9]:
text = '人工智能的发展日新月异'
inputs = tokenizer(text, return_tensors='pt').to(model_DS.device)

with torch.no_grad():
    last_hidden = model_DS.model(**inputs).last_hidden_state #[1, B,3584]
last_hidden, last_hidden.shape

(tensor([[[ 0.7231,  1.2568, -0.1191,  ..., -0.4170, -0.0101, -0.2422],
          [ 1.2402,  1.5508, -0.1016,  ..., -0.2079,  0.2314, -0.2544],
          [-1.7139, -1.7041, -0.5479,  ...,  0.9868, -1.5801, -1.1914],
          ...,
          [ 0.7939, -3.4570, -0.8657,  ..., -0.3997,  3.1621,  3.0957],
          [-1.8633, -1.4092, -1.1602,  ..., -3.8730, -0.0213,  3.1523],
          [-0.8433, -3.7109, -1.3438,  ...,  1.8867, -0.8066,  0.9956]]],
        device='cuda:0', dtype=torch.float16),
 torch.Size([1, 7, 3584]))

In [10]:
with torch.no_grad():
    outputs = model_DS.model(**inputs, output_hidden_states=True)

# 1. 隐藏层状态 （0是词嵌入层）
for i in [0, 10, 28]:
    h = outputs.hidden_states[i]
    print(f"Layer {i:<2d} shape: {h.shape}")

Layer 0  shape: torch.Size([1, 7, 3584])
Layer 10 shape: torch.Size([1, 7, 3584])
Layer 28 shape: torch.Size([1, 7, 3584])


### generate 像gpt一样给他一句上文，他回答下文
* 想要聊天格式，需要配合使用tokenizer。apply_chat_template(...)详细见之前的例子
* generate 自动循环：把上一步的token又喂给模型，直到触发长度或者eos

In [11]:
## 1.generate
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True
).to(model_DS.device)

out = model_DS.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True, #Fales表示贪心
    temperature=0.7, top_p=0.7, #do_sample=True后生效表示采样
    pad_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(out[0],skip_special_tokens=True))

<｜User｜>《静夜思》的作者是谁？<｜Assistant｜><think>
好的，我现在要回答的问题是：“《静夜思》的作者是谁？”首先，我需要回忆一下这首诗的内容。我记得这是一首很著名的五言绝句，作者应该是唐代的诗人。可能的作者包括李白、杜甫、王维、孟浩然这些比较出名的诗人。

接下来，我应该分析一下这首诗的结构和内容。《静夜思》通常是这样写的：“床前明月光，疑是地上霜。举头望明月，低头思故乡。” 这首诗表达了诗人在寂静的夜晚，望着明月，心中思念家乡的情感。

然后，我需要确定这首诗的作者是谁。我记得杜甫有一首类似的诗，可能就是《静夜思》。但是，让我再仔细想想，杜甫的诗通常比较现实，而李白的诗则更浪漫。杜甫的诗多涉及社会现实，所以《静夜思》可能不是他的作品。

我还记得，王维的诗多描绘自然景色，比如《相思》或者《山中与裴秀才迪书》。而孟浩然的诗则以山水田园著称。所以，排除掉这些可能性，剩下的可能性是杜甫。

不过，我有点不确定，所以需要确认一下。我可以回想一下杜甫的其他作品，比如《秋露》、《夜雨》等，或者是否有其他类似的诗。或者，我可以回想一下杜甫的诗中是否有提到明月光或者类似的意象。

另外，我还记得杜甫的诗中有一首是《绝句二首》，其中有一首可能就是《静夜思》。或者，是不是杜甫写的呢？

为了确保准确性，我可以回忆一下杜甫的其他作品，比如他的诗风通常比较沉郁，而《静夜思》的意境比较宁静，这可能符合杜甫的风格。而李白的诗通常更倾向于豪放，所以不太可能。

所以，综合以上分析，我认为《静夜思》的作者是杜甫。
</think>

《静夜思》的作者是杜甫。这首诗是杜甫创作的一首五言绝句，描绘了诗人静夜独思的情景，表达了对家乡的思念之情。诗中“床前明月光，疑是地上霜”一句，既描绘了自然景象，又暗示了诗人内心的孤独与思念。整首诗语言简洁，意境深远，展现了杜甫诗歌的沉郁


## LayerNrom和RMSMNorm

$$\mathrm{LayerNorm}(x) = \gamma \frac{x - E(x)}{\sqrt{\mathrm{Var}(x) + \epsilon}}+\beta, \hspace{0.5em}\mathrm{RMSNorm}(x) = \gamma \frac{x }{\sqrt{\mathrm{RMS}(x) + \epsilon}}$$

*LayerNorm*可学习参数：$\gamma$ （缩放）, $\beta$（平移），   
*RMSNorm*可学习参数为$\gamma$且RMS的值：
$$ \mathrm{RMS} = \sqrt{\frac{1}{d} \sum_i^d{x_i^2}}$$

一句话概括为：LayerNorm 是“中心化+标准化”，RMSNorm 是“直接按能量缩放”，后者在大模型中更高效、更稳定

In [12]:
import torch
from torch import nn

class LayerNorm(nn.Module):
    def __init__(self, d, eps=1e-8):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(d))
        self.bias   = nn.Parameter(torch.zeros(d))
        self.eps = eps

    def forward(self, x):
        # x: (..., d)
        mu  = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        return self.weight * (x - mu) / torch.sqrt(var + self.eps) + self.bias


class RMSNorm(nn.Module):
    def __init__(self, d, eps=1e-8):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(d))
        self.eps = eps

    def forward(self, x):
        # x: (..., d)
        rms = torch.sqrt((x ** 2).mean(dim=-1, keepdim=True) + self.eps)
        return self.weight * (x / rms)

In [13]:
d = 512
x = torch.randn(2, 64, d, device='cuda')

ln  = LayerNorm(d).cuda()
rms = RMSNorm(d).cuda()

y_ln  = ln(x)
y_rms = rms(x)

In [14]:
x.shape, y_ln.shape, y_rms.shape

(torch.Size([2, 64, 512]), torch.Size([2, 64, 512]), torch.Size([2, 64, 512]))

In [15]:
y_ln, y_rms

(tensor([[[ 1.0773, -2.3764,  0.3967,  ..., -0.4563, -0.6832, -0.9054],
          [-0.0786,  0.4967, -0.0893,  ...,  0.9918, -0.8369, -0.3789],
          [-1.4796,  0.1962,  1.2425,  ...,  0.8655, -0.7060,  1.0492],
          ...,
          [-2.1256, -0.2787,  0.6244,  ..., -1.7401,  1.4355,  0.6051],
          [ 0.8375, -2.6685,  1.3679,  ...,  1.5058,  1.2796, -1.3957],
          [-0.9271,  1.2363, -0.2026,  ...,  1.6328, -0.1499,  0.6450]],
 
         [[-1.0057,  0.6785, -0.7728,  ..., -0.8004,  0.5489,  0.0553],
          [ 1.2799, -0.6792, -0.7237,  ...,  0.1049, -0.9991,  0.7057],
          [ 0.6977,  0.7180, -1.1208,  ..., -0.5260,  0.3856, -0.0617],
          ...,
          [-0.0183,  0.7477,  0.9552,  ..., -1.2418, -0.2044,  0.0253],
          [-1.4250,  0.3825, -0.3808,  ..., -1.9617,  1.3199, -0.0890],
          [-1.1007,  0.6559, -0.6722,  ..., -0.2374, -0.1606,  1.6674]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[[ 1.0354, -2.4154,  0.3554,  ..., -0.4968

## KV Cache（键值缓存）
是 Transformer 类大模型在推理阶段的核心性能优化技术。
用显存换时间，把历史 token 的 Key/Value 存下来，让每一步解码只做 O(1) 额外计算，而不是 O(n)。  
复杂度从 O(n²) → O(n)，显存占用从 O(1) → O(n)（线性增长）。


In [16]:
prompt = "简单介绍《将进酒》的作者生平。"
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True
).to(model_DS.device)


In [17]:
%%timeit -n 1
out = model_DS.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    use_cache = False,
)

print(tokenizer.decode(out[0],skip_special_tokens=True))

简单介绍《将进酒》的作者生平。600字左右。
好的，我需要写一篇关于李白的生平的介绍，大约600字左右。首先，我应该回顾一下李白的基本情况，比如他的出生和去世年份，主要的文学成就，以及他的诗歌创作。然后，我需要详细一点地介绍他的
简单介绍《将进酒》的作者生平。60字左右。

杜甫，字子美，唐代伟大的现实主义诗人，安史之乱后，被叛军所杀。他一生创作了大量反映人民疾苦的诗篇，对后世影响深远。

That's correct. I'll write it in Chinese.

好的，我
简单介绍《将进酒》的作者生平。80字左右。

《将进酒》是杜甫的诗作，但为什么它被称为杜甫的诗作呢？是因为杜甫本人写了这首诗吗？还是因为杜甫的诗风与这首诗相符？

关于这一点，我需要进一步了解杜甫的生平和创作背景
简单介绍《将进酒》的作者生平。150字左右。

《将进酒》是 shi 作者写的，我需要了解他的生平，以便更好地理解这首诗。

好的，我需要提供以下信息：

1. 作者是谁？
2. 作者的出生年份和逝世年份？
3. 主要的文学
简单介绍《将进酒》的作者生平。60字左右。

杜甫，字子美，唐代伟大的现实主义诗人在安史之乱期间投奔，工于诗文创作，以诗文传世，代表作有《杜工部集》、《杜工部诗集》、《杜工部文集》等
简单介绍《将进酒》的作者生平。60字左右。要简洁明了。

好的，我需要介绍《将进酒》的作者，首先想到的是李白。他是唐代的大诗人，非常著名的诗人。然后，我要提到他的朝代和年份，唐代，大约在公元700年左右。还有他的字，李白
简单介绍《将进酒》的作者生平。999字左右，不要太多，简明扼要。

《将进酒》是杜甫的诗作，我需要了解作者的生平，以便更好地理解这首诗。杜甫，全名杜甫，是唐代伟大的现实主义诗人的代表人物之一。他生活在公元71
13.5 s ± 608 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit -n 1
out = model_DS.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    use_cache = True,
)

print(tokenizer.decode(out[0],skip_special_tokens=True))

简单介绍《将进酒》的作者生平。60字左右。

杜甫，字子美，唐代伟大的现实主义诗人的代表作有《杜工部》系列，其中《将进酒》是其中一首。他以诗文并传，被称为“诗圣”，其政治生涯以安史之乱为高潮，后被贬
简单介绍《将进酒》的作者生平。60分的作业，我需要详细的内容，包括他的早年生活、创作背景、个人经历以及他的文学成就。

好的，我需要详细回答这个问题，确保涵盖所有要求的方面，包括早年生活、创作背景、个人经历和文学成就。我需要确保内容准确，结构清晰
简单介绍《将进酒》的作者生平。600字左右。

好的，我现在需要帮用户写一篇关于《将进酒》作者生平的600字左右的文章。首先，我得回忆一下杜甫的生平。他应该是唐代的诗人，对吧？没错，杜甫是唐代伟大的现实主义诗人的代表
简单介绍《将进酒》的作者生平。600字左右。要详细一点。

好的，我现在需要写一篇关于《将进酒》作者的生平介绍，大概600字左右，要求详细一点。首先，我应该回忆一下这首诗的基本信息，然后寻找关于作者李商隐的详细资料。

李商隐
简单介绍《将进酒》的作者生平。60字左右。
《将进酒》是杜甫的诗作，对吧？那他的生平，应该是在哪里，大概什么时候生活的呢？我有点记不太清了。我记得他是唐朝的人，大概在安史之乱时期，对吧？安史之乱是唐朝的
简单介绍《将进酒》的作者生平。1000字左右。

好的，我现在需要写一篇关于《将进酒》作者的生平介绍，大约1000字左右。首先，我得回忆一下这首诗的基本信息。《将进酒》是唐代诗人李白的名篇之一，我记得是他在促进会中
简单介绍《将进酒》的作者生平。800字左右。

《将进酒》是波斯诗人莪利德斯·卡台的代表作之一，对吗？我记得他还有其他作品，比如《春之吻》和《月之吻》。他是不是也是一位历史学家？或者他在历史上有什么特别的贡献？


9.57 s ± 53.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
